In [ ]:
%run includes/3-network-setup.ipynb

In [ ]:

# get networks
queries = [
    f"""
    SELECT name,
    '{project}' as project,
    mtu,
    networkFirewallPolicyEnforcementOrder as fwPolicyEnforcementOrder,
    JSON_ARRAY_LENGTH(peerings) as num_peerings,
    peerings,
    JSON_EXTRACT(routingConfig, '$.routingMode') as routingMode,
    subnetworks
    FROM google.compute.networks WHERE project = '{project}'
    AND name IS NOT NULL
    """
    for project in compute_enabled_projects
]

networks_df = run_stackql_queries(queries)
networked_projects = list(networks_df['project'].unique())
networks_df

In [ ]:

# network peerings
networks_subset_df = networks_df[networks_df['peerings'] != 'null'][['name', 'peerings']].copy()
network_peerings_df = explode_json_list_col(networks_subset_df, 'peerings', 'peering')
network_peerings_df

In [ ]:

# subnetworks
queries = [
    f"""
    SELECT 
    name,
    SPLIT_PART(region, '/', 9) as region,
    SPLIT_PART(network, '/', 10) as network,
    stackType,
    ipCidrRange,
    secondaryIpRanges,
    gatewayAddress,
    ipv6CidrRange,
    purpose,
    enableFlowLogs,
    privateIpGoogleAccess
    FROM google.compute.subnetworks WHERE project = '{project}'
    """
    for project in networked_projects
]

subnetworks_df = run_stackql_queries(queries)
subnetworks_df

In [ ]:

# get interconnects
queries = [
    f"""
    SELECT *
    FROM google.compute.interconnects WHERE project = '{project}'
    AND name IS NOT NULL
    """
    for project in networked_projects
]

interconnects_df = run_stackql_queries(queries, True)
interconnects_df

In [ ]:

# get vpns
queries = [
    f"""
    SELECT 
    name,
    '{project}' as project, 
    SPLIT_PART(network, '/', 9) as network,
    SPLIT_PART(region, '/', 8) as region,
    vpnInterfaces
    FROM google.compute.vpn_gateways WHERE project = '{project}'
    """
    for project in networked_projects
]

vpns_df = run_stackql_queries(queries, True)
vpns_df

In [ ]:

# Convert the string to list of dictionaries and explode
exploded_df = vpns_df.assign(vpnInterfaces=vpns_df['vpnInterfaces'].apply(ast.literal_eval)).explode('vpnInterfaces')
exploded_df[['id', 'ipAddress']] = exploded_df['vpnInterfaces'].apply(pd.Series)
exploded_df.drop(columns=['vpnInterfaces'], inplace=True)
exploded_df

In [ ]:

# get addresses
queries = [
    f"""
    SELECT 
    id,
    name,
    '{project}' as project, 
    SPLIT_PART(network, '/', 9) as network,
    SPLIT_PART(subnetwork, '/', 11) as subnetwork,
    addressType,
    ipVersion,
    ipv6EndpointType,
    purpose,
    labels,
    status,
    users,
    networkTier
    FROM google.compute.addresses WHERE project = '{project}'
    """
    for project in networked_projects
]

addresses_df = run_stackql_queries(queries, True)
addresses_df